# Import packages

In [1]:
import math, os, collections
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from evaluation import Evaluation
from db_connection.utils import get_conn, get_data_start_dt
from utils import load_w103, load_w106, load_cust, cust_process
from mlaas_tools.config_build import config_set

In [2]:
## Configure env
if not os.path.isfile('config.ini'):
    config_set()

In [3]:
## Load db connection
rawdata_conn = get_conn('edu')

read key file
login as edu-cytsao


In [5]:
rawdata_conn = get_conn('edu')
# interaction train w103
w103_df = load_w103('2019-01-31', rawdata_conn, span=18)
w103_df['wm_prod_code'].value_counts()

read key file
login as edu-cytsao


AG25    83106
KK32    75595
CC03    54323
AA39    48277
CC88    40423
        ...  
AC10        1
8947        1
2930        1
UV05        1
8928        1
Name: wm_prod_code, Length: 2263, dtype: int64

In [10]:
w103_df.groupby("wm_prod_code")["wm_prod_code"].count().sort_values(ascending=False).head(5).index.to_list()

['AG25', 'KK32', 'CC03', 'AA39', 'CC88']

In [20]:
w103_df.groupby(["cust_no","wm_prod_code"])['wm_prod_code'].value_counts()[:5]#.index.to_list()

cust_no                   wm_prod_code  wm_prod_code
++++2rDzc1I9amUGBhmxLA==  5821          5821             1
++1QLn8bCFzxWN8vQYBuRg==  NN18          NN18            32
++2WLG2lyrgp9jhLkCFEKg==  2509          2509            18
                          AG74          AG74            18
                          WW06          WW06            18
Name: wm_prod_code, dtype: int64

In [20]:
top5_fund = w103_df.groupby(["cust_no","wm_prod_code"])["wm_prod_code"].count().sort_values(ascending=False).head(5).index.to_list()
top5_fund = [i[1] for i in top5_fund]
top5_fund

['PP20', 'BB51', 'AG02', 'KK41', 'CC79']

In [34]:
top_dict = {i:0 for i in top5_fund}
top_dict

{'PP20': 0, 'BB51': 0, 'AG02': 0, 'KK41': 0, 'CC79': 0}

In [13]:
len(w103_df.groupby(["cust_no","wm_prod_code"]))

192987

In [18]:
print(len(w103_df), len(w103_df.cust_no.unique()))

1950007 72389


In [15]:
w103_df.groupby(["cust_no","wm_prod_code"], group_keys=False).apply(lambda x: x.sort_values(["deduct_cnt"],ascending=True).head(1))

,cust_no,wm_prod_code,txn_dt,txn_amt,dta_src,deduct_cnt,etl_dt
90556,++++2rDzc1I9amUGBhmxLA==,5821,2017-11-27,1113229.0,,0.0,2018-09-25
54929,++1QLn8bCFzxWN8vQYBuRg==,NN18,2017-08-08,29000.0,,3.0,2018-09-25
95384,++2WLG2lyrgp9jhLkCFEKg==,2509,2017-08-14,29000.0,,122.0,2018-09-25
95394,++2WLG2lyrgp9jhLkCFEKg==,AG74,2017-08-17,29000.0,,124.0,2018-09-25
95380,++2WLG2lyrgp9jhLkCFEKg==,WW06,2017-08-17,29000.0,,116.0,2018-09-25
...,...,...,...,...,...,...,...
1949729,zzvYlT/G3nOSg2SdheESWg==,AT07,2018-02-07,1021728.0,A0,0.0,2018-09-25
1949723,zzvYlT/G3nOSg2SdheESWg==,AV03,2018-02-12,17011.0,,1.0,2018-09-25
1949731,zzvYlT/G3nOSg2SdheESWg==,BB39,2017-11-02,870000.0,A0,0.0,2018-09-25
1949739,zzvYlT/G3nOSg2SdheESWg==,FE41,2017-11-02,870000.0,,0.0,2018-09-25


In [21]:
w103_df.groupby("wm_prod_code")["wm_prod_code"].count().sort_values(ascending=False).head(5).to_dict()

{'AG25': 83106, 'KK32': 75595, 'CC03': 54323, 'AA39': 48277, 'CC88': 40423}

## Load data

In [5]:
today = '2018-12-31'
span=1
duration = '1m'
## Load db connection
rawdata_conn = get_conn('edu')
## Load data
print("Loading Data...")
# interaction train w103
w103_df = load_w103(today, rawdata_conn, span)
# user
cust_df = load_cust(today, rawdata_conn, span)
# intersection
user_filter = set(w103_df['cust_no'].tolist()) & set(cust_df['cust_no'].tolist())
w103_df = w103_df[w103_df['cust_no'].isin(user_filter)]
cust_df_filter = cust_df[cust_df['cust_no'].isin(user_filter)] 
purchase_hist = w103_df.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))).to_dict()
# evaluation w103
evaluation = Evaluation(today, None, duration, purchase_hist)
evaluate_w103 = evaluation.read(today, rawdata_conn, duration)
# evaluation user features
eva_cust_df = load_cust(today, rawdata_conn, span=None, mode='evaluation')
# intersection
eva_user_filter = set(evaluate_w103['cust_no'].tolist()) & set(eva_cust_df['cust_no'].tolist())
eva_cust_df_filter = eva_cust_df[eva_cust_df['cust_no'].isin(eva_user_filter)]
evaluate_w103 = evaluate_w103[evaluate_w103['cust_no'].isin(eva_user_filter)]

read key file
login as edu-cytsao
Loading Data...
loading 2018-12-01 2018-12-31 data.
read key file
login as edu-cytsao
loading 2019-01-01 2019-01-31 data.


In [6]:
warm_users, cold_users = evaluation.warm_cold_list()
print('warm-users:', len(warm_users), 'cold-users:', len(cold_users))

# cold start users
user_list = evaluate_w103[evaluate_w103['cust_no'].isin(cold_users)]['cust_no'].unique().tolist()
print(len(user_list))

warm-users: 1517 cold-users: 2366
2334


In [11]:
eva_cust_df_cold = eva_cust_df[eva_cust_df['cust_no'].isin(user_list)]
eva_cust_df_cold

,cust_no,age,gender_code,cust_vintage,income_range_code,risk_type_code,children_cnt,edu_code,wm_club_class_code
4,00ugL9CAflI4eVOrkUVKDg==,3.0,M,2.0,1,03,0.0,6,O
26,02wYCJa+FZ51NXzlZx6oBQ==,3.0,F,2.0,3,04,0.0,4,P
30,03oAsRwG9EttbqNCNYqmkQ==,1.0,F,0.0,1,03,0.0,4,0
38,03ZyLI0fXpZ7Nor9qNgK0w==,1.0,F,0.0,1,04,0.0,4,0
84,08j1Y6cVBuiPJY4T1YLOxA==,3.0,F,0.0,1,03,3.0,6,0
...,...,...,...,...,...,...,...,...,...
35555,zxZd5h9bkBvTYcUxev6lXQ==,3.0,M,2.0,2,03,0.0,3,M
35593,+zyVGaPHp1w9MVYLHGQPSA==,3.0,M,0.0,2,04,0.0,6,Q
35611,ZzfPSKfjdZnv5yyYg3YHHA==,2.0,F,1.0,1,02,0.0,3,0
35615,zZijdmkMLP9riddUbF778A==,3.0,M,1.0,1,03,0.0,3,0


In [12]:
cust_new_matrix = eva_cust_df_cold.iloc[:, 1:].values
cust_new_matrix

array([[3.0, 'M', 2.0, ..., 0.0, '6', 'O'],
       [3.0, 'F', 2.0, ..., 0.0, '4', 'P'],
       [1.0, 'F', 0.0, ..., 0.0, '4', '0'],
       ...,
       [2.0, 'F', 1.0, ..., 0.0, '3', '0'],
       [3.0, 'M', 1.0, ..., 0.0, '3', '0'],
       [1.0, 'F', 0.0, ..., 0.0, '2', '0']], dtype=object)

In [24]:
cat_cols = ['gender_code', 'income_range_code','risk_type_code', 'edu_code', 'wm_club_class_code']
eva_cust_df_cold_dum = pd.get_dummies(eva_cust_df_cold, columns=cat_cols)
eva_cust_df_cold_dum

,cust_no,age,cust_vintage,children_cnt,gender_code_F,gender_code_M,income_range_code_1,income_range_code_2,income_range_code_3,income_range_code_4,...,wm_club_class_code_2,wm_club_class_code_3,wm_club_class_code_F,wm_club_class_code_H,wm_club_class_code_M,wm_club_class_code_N,wm_club_class_code_O,wm_club_class_code_P,wm_club_class_code_Q,wm_club_class_code_Z
4,00ugL9CAflI4eVOrkUVKDg==,3.0,2.0,0.0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26,02wYCJa+FZ51NXzlZx6oBQ==,3.0,2.0,0.0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
30,03oAsRwG9EttbqNCNYqmkQ==,1.0,0.0,0.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,03ZyLI0fXpZ7Nor9qNgK0w==,1.0,0.0,0.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,08j1Y6cVBuiPJY4T1YLOxA==,3.0,0.0,3.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35555,zxZd5h9bkBvTYcUxev6lXQ==,3.0,2.0,0.0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
35593,+zyVGaPHp1w9MVYLHGQPSA==,3.0,0.0,0.0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
35611,ZzfPSKfjdZnv5yyYg3YHHA==,2.0,1.0,0.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35615,zZijdmkMLP9riddUbF778A==,3.0,1.0,0.0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
cust_new_matrix = eva_cust_df_cold_dum.iloc[:, 1:].values
cust_new_matrix.shape

(2334, 31)

In [28]:
cust_old_matrix = pd.get_dummies(cust_df_filter, columns=cat_cols).iloc[:, 1:].values
cust_old_matrix.shape

(34317, 35)

### Align cols

In [32]:
all_dummy = pd.get_dummies(pd.concat([cust_df_filter, eva_cust_df_cold]), columns=cat_cols)
all_dummy

,cust_no,age,cust_vintage,children_cnt,gender_code_F,gender_code_M,income_range_code_1,income_range_code_2,income_range_code_3,income_range_code_4,...,wm_club_class_code_G,wm_club_class_code_H,wm_club_class_code_I,wm_club_class_code_J,wm_club_class_code_M,wm_club_class_code_N,wm_club_class_code_O,wm_club_class_code_P,wm_club_class_code_Q,wm_club_class_code_Z
0,00FmcV87beY8s7OI0Xjtkg==,2.0,2.0,0.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,00IrxuhcWjiKQPXJDBv2Yg==,2.0,2.0,0.0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00qPHDE+5ykU4RA/Ce18Pw==,3.0,1.0,0.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/00rG/y1/402L1woM8tdmA==,2.0,2.0,0.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00roz6dL8l5Ouk54dlOc9A==,3.0,2.0,0.0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35555,zxZd5h9bkBvTYcUxev6lXQ==,3.0,2.0,0.0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
35593,+zyVGaPHp1w9MVYLHGQPSA==,3.0,0.0,0.0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
35611,ZzfPSKfjdZnv5yyYg3YHHA==,2.0,1.0,0.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35615,zZijdmkMLP9riddUbF778A==,3.0,1.0,0.0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
cust_old_matrix = all_dummy.iloc[:len(cust_old_matrix)].iloc[:, 1:].values
cust_old_matrix.shape

(34317, 35)

In [74]:
old_cust_idx = cust_df_filter.reset_index().cust_no.to_dict()
print(len(old_cust_idx))
print(max(old_cust_idx.keys()))

34317
34316


In [36]:
cust_new_matrix = all_dummy.iloc[len(cust_old_matrix):].iloc[:, 1:].values
cust_new_matrix.shape

(2334, 35)

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(cust_new_matrix, cust_old_matrix)

In [44]:
sim

array([[0.7190925 , 0.78446454, 0.73029674, ..., 0.84983659, 0.31622777,
        0.81989159],
       [0.78446454, 0.7190925 , 0.85201287, ..., 0.7190925 , 0.10540926,
        0.59628479],
       [0.56613852, 0.45291081, 0.63245553, ..., 0.67936622, 0.36514837,
        0.64549722],
       ...,
       [0.61394061, 0.78935222, 0.73484692, ..., 0.78935222, 0.28284271,
        0.8       ],
       [0.64450339, 0.85933785, 0.73333333, ..., 0.78772636, 0.34641016,
        0.9797959 ],
       [0.33968311, 0.56613852, 0.63245553, ..., 0.56613852, 0.54772256,
        0.51639778]])

In [51]:
def fast_topK(sim, K):
    """取相似度TopK的顧客"""
    partial_sorting_sim = np.argpartition(-sim, K, 1)
    col_index = np.arange(sim.shape[0])[:, None]
    argsort_K = np.argsort(sim[col_index, partial_sorting_sim[:, 0:K]], 1)
    return (partial_sorting_sim[:, 0:K][col_index, argsort_K]).tolist()

In [57]:
test = np.random.randint(1, 5, size=(2, 3))
print(test)
fast_topK(test, 2)

[[2 2 3]
 [3 3 2]]


[[1, 2], [0, 1]]

In [76]:
topk_sim_old_user = fast_topK(sim, 5)
for i in range(len(topk_sim_old_user)):
    topk_sim_old_user[i] = [old_cust_idx[j] for j in topk_sim_old_user[i]]
topk_sim_old_user[:2]

[['EXgrxSNHQpuBuVUUtQ/+nw==',
  '4P14c35KNG482rmUf8/8CQ==',
  'vD8bZWyU8tHaiOPG6bHVJw==',
  'SrqlzCkosjUpRtDPUfxUqA==',
  '9exbiO6wBafQ7PNHA0S8FQ=='],
 ['BtXsUo2n5cBcnlmMPnEH4g==',
  'SF3lMEyEXDlSPdUiXSjBeg==',
  'OcNE/7Yv9EL9+arnzMWa1g==',
  'UKHlr3sOUd5MVFYpNvICgg==',
  'sIFCj1PPdZOnPDDz2rkfZA==']]

In [77]:
recommend_funds = []
for i in topk_sim_old_user:
    hist_fund = []
    for old_u in i:
        hist_fund += purchase_hist[old_u]
    recommend_funds.append(hist_fund[:5])
print(recommend_funds[:2])

[['AG25', 'BB39', 'BB46', 'NN47', 'AG24'], ['6802', 'AG15', 'CC97', 'KK32', 'AB04']]


In [78]:
len(recommend_funds)

2334